In [ ]:
!pip install yfinance
!pip install prophet
!pip install pystan

# Import necessary libraries
from prophet import Prophet
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from prophet.diagnostics import performance_metrics, cross_validation
from prophet.plot import plot_plotly, plot_components_plotly


# Download SPX ticker data
ticker="^SPX"
data = yf.download(ticker, start="2003-10-01", end="2023-10-01")

# Prepare the data for Prophet model
df = pd.DataFrame()
df["ds"] = data.index
df["y"] = data["Adj Close"].values
df.dropna(inplace=True)

# Create and train Prophet model
model = Prophet(
    seasonality_mode="multiplicative",
    changepoint_prior_scale=0.01,
    seasonality_prior_scale=0.01,
    n_changepoints=500,
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    growth="linear",
    holidays=None,
    ).add_seasonality(
        name='monthly',
        period=30.5,
        fourier_order=10
    )


# Fit the model to the data
model.fit(df)

# Create a dataframe for making future predictions
future = model.make_future_dataframe(periods=5)  # Extend the time frame by 5 days

# Predict future values
forecast = model.predict(future)


# Plot the forecast
fig = model.plot(forecast)
plt.title("S&P 500 Index Forecast")
plt.xlabel("Date")
plt.ylabel("Value")
fig.gca().get_lines()[0].set_color("blue")  # Actual data (blue)
fig.gca().get_lines()[1].set_color("orange")  # Forecasted data (orange)
plt.legend(["Actual Data", "Forecasted Data"], loc="upper left")
plt.show()

# Access prediction components (trend, seasonality, holidays)
forecast_components = model.plot_components(forecast)
plt.suptitle("Forecast Components")
plt.subplots_adjust(top=0.9)  # Adjust the top margin for the title
plt.show()

# Calculate and display performance metrics (e.g., RMSE, MAE)
df_cv = cross_validation(model, horizon='5 days', period='365 days', initial='3650 days')
performance = performance_metrics(df_cv)

# Create a bar plot for performance metrics
performance.plot(x='horizon', y='mape', kind='bar', color='b', legend=False)
plt.title("Performance Metrics")
plt.xlabel("Forecast Horizon")
plt.ylabel("Mean Absolute Percentage Error (MAPE)")
plt.show()

print(performance)